In [1]:
!pip install nltk

     |████████████████████████████████| 1.4 MB 1.1 MB/s eta 0:00:01
     |████████████████████████████████| 82 kB 639 kB/s eta 0:00:01
     |████████████████████████████████| 723 kB 1.1 MB/s eta 0:00:01
     |████████████████████████████████| 69 kB 869 kB/s eta 0:00:01
  Created wheel for nltk: filename=nltk-3.5-py3-none-any.whl size=1435512 sha256=a7868cf4a5ff448ef07f33926432a97dfee5ec0da326d2f8c53acf345a3dbfe9
  Stored in directory: /root/.cache/pip/wheels/de/5e/42/64abaeca668161c3e2cecc24f864a8fc421e3d07a104fc8a51
Successfully built nltk
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [2]:
import pdb
import pickle
import string
import time
import matplotlib.pyplot as plt
import nltk
import numpy as np
import scipy
from nltk.corpus import stopwords, twitter_samples
from collections import defaultdict
from nltk.tokenize import TweetTokenizer
from os import getcwd
import re
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk import pos_tag
from collections import defaultdict
from nltk.stem import PorterStemmer

In [3]:
en_embeddings_subset = pickle.load(open("../data/en_embeddings.p", "rb"))

In [4]:
filePath = f"{getcwd()}/data/"
nltk.data.path.append(filePath)

In [5]:
filePath

'/tf/NLP/NLP/NLP with Classification and Vector Spaces/data/'

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('omw')

In [6]:
# tags needed for lemmatization
tag_dict = defaultdict(lambda : wordnet.NOUN)
tag_dict['J'] = wordnet.ADJ
tag_dict['V'] = wordnet.VERB
tag_dict['R'] = wordnet.ADV

def process_tweet(text):
    
    tweet = text.lower()
    tweet = re.sub(r'[0-9]+', '', tweet)
    # remove stock market tickers like $GE
    tweet = re.sub(r'\$\w*', '', tweet)
    # remove old style retweet text "RT"
    tweet = re.sub(r'^RT[\s]+', '', tweet)
    # remove hyperlinks
    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)
    # remove hashtags
    # only removing the hash # sign from the word
    tweet = re.sub(r'#', '', tweet)
    
    tweet = tweet.translate((None, string.punctuation))
    
  
    #tokenize tweets
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True,
                               reduce_len=True)
    tweet_tokens = tokenizer.tokenize(tweet)
    stopwords_english = stopwords.words('english')
    tweet = [word for word in tweet_tokens if word not in stopwords_english]
    stemmer = PorterStemmer()
    tweets_clean = []
    
    lemmatizer=WordNetLemmatizer()
    lemmatize_text = []
    for token, tag in pos_tag(tweet):
        word = lemmatizer.lemmatize(token, tag_dict[tag[0]])
        stem_word = stemmer.stem(word)  # stemming word
        if(len(stem_word)>1):
            tweets_clean.append(stem_word)
    
    return tweets_clean

LookupError: 
**********************************************************************
  Resource [93mwordnet[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('wordnet')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/wordnet[0m

  Searched in:
    - '/root/nltk_data'
    - '/usr/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
    - '/tf/NLP/NLP/NLP with Classification and Vector Spaces/data/'
**********************************************************************


In [ ]:
def get_document_embedding(doc,embedding):
    
    doc_embedding = np.zeros(300)
    clean_doc = process_tweet(doc)
    for word in clean_doc:
        doc_embedding += embedding.get(word,0)
        
    return doc_embedding

In [ ]:
def get_document_vecs(all_docs, en_embeddings):
  
    ind2Doc_dict = {}
    document_vec_l = []
    for i, doc in enumerate(all_docs):
        doc_embedding = get_document_embedding(doc,en_embeddings)
        ind2Doc_dict[i] = doc_embedding

        document_vec_l.append(doc_embedding)
    document_vec_matrix = np.vstack(document_vec_l)

    return document_vec_matrix, ind2Doc_dict

In [ ]:
document_vecs, ind2Tweet = get_document_vecs(all_tweets, en_embeddings_subset)

In [ ]:
N_VECS = len(all_tweets)       # This many vectors.
N_DIMS = len(ind2Tweet[1])     # Vector dimensionality.
print(f"Number of vectors is {N_VECS} and each has {N_DIMS} dimensions.")

### use LSH for partitioning documents vector space to search easier

Each plane divides the space to $2$ parts.

So $n$ planes divide the space into $2^{n}$ hash buckets.

We want to organize 10,000 document vectors into buckets so that every bucket has about $~16$ planes.
For that we need $\frac{10000}{16}=625$ buckets.

We're interested in $n$, number of planes, so that $2^{n}= 625$. Now, we can calculate $n=\log_{2}625 = 9.29 \approx 10$.

In [ ]:
# The number of planes. We use log2(256) to have ~16 vectors/bucket.
N_PLANES = 10
# Number of times to repeat the hashing to improve the search.
N_UNIVERSES = 25

In [ ]:
np.random.seed(0)
planes_l = [np.random.normal(size=(N_DIMS, N_PLANES)) for _ in range(N_UNIVERSES)]

In [ ]:
def hash_value_of_vector(v, planes):
   
    dot_product = np.dot(v,planes)
    # get the sign of the dot product (1,10) shaped vector
    sign_of_dot_product = np.sign(dot_product)
    
    # set h to be false (eqivalent to 0 when used in operations) if the sign is negative,
    # and true (equivalent to 1) if the sign is positive (1,10) shaped vector
    h = sign_of_dot_product>=0

    # remove extra un-used dimensions (convert this from a 2D to a 1D array)
    h = np.squeeze(h)

    # initialize the hash value to 0
    hash_value = 0

    n_planes = planes.shape[1]
    for i in range(n_planes):
        # increment the hash value by 2^i * h_i
        hash_value += np.power(2,i)*h[i]

    # cast hash_value as an integer
    hash_value = int(hash_value)

    return hash_value

In [ ]:
def make_hash_table(v,plans):
    
    num_of_planes = planes.shape[1]
    num_buckets = 2^num_of_planes
    hash_table={i:[] for i in range(num_buckets)}
    
    for i, v in enumerate(vecs):
        h = hash_value_of_vector(v,planes)
        hash_table[h].append(v)
        
    return hash_table

In [ ]:
hash_tables = []
for universe_id in range(N_UNIVERSES):  # there are 25 hashes
    print('working on hash universe #:', universe_id)
    planes = planes_l[universe_id]
    hash_table = make_hash_table(document_vecs, planes)
    hash_tables.append(hash_table)


In [ ]:
def nearest_neighbor(v, candidates, k=1):
   
    similarity_l = []

    for row in candidates:
        cos_similarity = cosine_similarity(v,row)
        similarity_l.append(cos_similarity)
        
    sorted_ids = np.argsort(similarity_l)

    k_idx = sorted_ids[-k:]
    return k_idx

In [ ]:
def approximate_knn(doc_id, v, planes_l, k=1, N_UNIVERSES,hash_tables):
    candidates = list()
    for universe_id in range(N_UNIVERSES):
        hash_value = hash_value_of_vector(v,planes_l[universe_id])
        hash_table = hash_tables[universe_id]
        bucket = hash_table[hash_value]
        
        if v in bucket:
            bucket.remove(v)
        for v in bucket:
            if v not in candidates:
                candidates.appenf(v)
                
    candidates_arr = np.array(candidates)
    nearest_neighbor_idx_l = nearest_neighbor(v, candidates_arr, k=k)
    
    return nearest_neighbor_ids